In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
# Load SQuAD -- Q&A dataset developed by crowdworkers on a set of Wikipedia articles.
# Here, the answer to every question is a segment of text from the corresponding reading passage / context.
from datasets import load_dataset

squad = load_dataset("squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [43]:
squad['train'][2]['context']

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [44]:
squad['train'][2]['question']

'The Basilica of the Sacred heart at Notre Dame is beside to which structure?'

In [45]:
squad['train'][2]['answers']

{'text': ['the Main Building'], 'answer_start': [279]}

In [50]:
example = squad['train'][1100]
for key in example:
  print(key, ":", example[key])

id : 57341b484776f41900661888
title : Montana
context : Montana is home to a diverse array of fauna that includes 14 amphibian, 90 fish, 117 mammal, 20 reptile and 427 bird species. Additionally, there are over 10,000 invertebrate species, including 180 mollusks and 30 crustaceans. Montana has the largest grizzly bear population in the lower 48 states. Montana hosts five federally endangered species–black-footed ferret, whooping crane, least tern, pallid sturgeon and white sturgeon and seven threatened species including the grizzly bear, Canadian lynx and bull trout. The Montana Department of Fish, Wildlife and Parks manages fishing and hunting seasons for at least 17 species of game fish including seven species of trout, walleye and smallmouth bass and at least 29 species of game birds and animals including ring-neck pheasant, grey partridge, elk, pronghorn antelope, mule deer, whitetail deer, gray wolf and bighorn sheep.
question : How many species of game fish have hunting seasons?


In [5]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForQuestionAnswering.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def prepare_training_features(examples):
    # Tokenize our examples with truncation and padding.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second", # truncate context, not the question
        max_length=384, # maximum length for the combined question and context
        stride=128, # when the context needs to be split into parts, stride is the number of overlapping topkens
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [7]:
# Apply the function to our data
tokenized_datasets = squad.map(prepare_training_features, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 88524
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 10784
    })
})

In [227]:
# Fine-tune the model
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"finetune-BERT-squad",
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [54]:
# How data is collated (batched) during training. It does pads sequences and creates input tensors.
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [228]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"].select(range(2000)),
    eval_dataset=tokenized_datasets["validation"].select(range(200)),
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-228-bc02bc43d9a1>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [229]:
import torch

trainer.train()

Step,Training Loss
500,0.186500


TrainOutput(global_step=625, training_loss=0.15572822341918946, metrics={'train_runtime': 791.066, 'train_samples_per_second': 12.641, 'train_steps_per_second': 0.79, 'total_flos': 1959725675520000.0, 'train_loss': 0.15572822341918946, 'epoch': 5.0})

In [243]:
# We will use the instance with index 0 in the SQuAD validation dataset as an example.
instance = squad['validation'][53]
context = instance['context']
question = instance['question']

In [244]:
instance

{'id': '56d98a59dc89441400fdb52b',
 'title': 'Super_Bowl_50',
 'context': 'The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl.',
 'question': 'Who did the Panthers beat to become the NFC champs?',
 'answers': {'text': ['Arizona Cardinals',
   'the Arizona Cardinals',
   'Arizona Cardinals'],
  'answer_start': [152, 148, 152]}}

In [245]:
given_answer = instance['answers']['text'][1] # Assuming the first answer is the correct one
given_answer_start = instance['answers']['answer_start'][0]
given_answer, given_answer_start

('the Arizona Cardinals', 152)

In [246]:
# Tokenize the data
inputs = tokenizer("Who did the Panthers beat to become the NFC champs?", context, return_tensors='pt',max_length=512, truncation=True)

In [247]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [248]:
# Extract specific fields and their values from that input
# Transfer all the input tensors to the GPU
# preparing them for processing by the model which is already on that device.
inputs = {key: val.to(device) for key, val in inputs.items()}

In [249]:
# Get model's output
with torch.no_grad():
    output = model(**inputs)

In [250]:
# Get the predicted answer
start_index = torch.argmax(output.start_logits)
end_index = torch.argmax(output.end_logits)

predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index + 1]))

In [251]:
predicted_answer, start_index.item(), end_index.item()

('arizona cardinals', 44, 45)

In [252]:
correct = (predicted_answer.lower() == given_answer.lower())
evaluation = 'Correct' if correct else f'Incorrect (Predicted: {predicted_answer}, Given: {given_answer})'

print(evaluation)

Incorrect (Predicted: arizona cardinals, Given: the Arizona Cardinals)


In [232]:
def evaluate_instance(instance, device):
    context = instance['context']
    question = instance['question']
    given_answer = instance['answers']['text'][0]  # Assuming the first answer is the correct one

    inputs = tokenizer(question, context, return_tensors='pt', max_length=512, truncation=True)

    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        output = model(**inputs)

    start_index = torch.argmax(output.start_logits)
    end_index = torch.argmax(output.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index + 1]))


    return predicted_answer.lower() == given_answer.lower()

In [193]:
# For creating progress bar
from tqdm import tqdm

In [241]:
correct_count = 0
total_count = 200

for i in tqdm(range(total_count)):
    correct_count += evaluate_instance(squad['train'][i], device)

100%|██████████| 200/200 [00:02<00:00, 68.84it/s]


In [242]:
accuracy = correct_count / total_count
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 82.50%
